First I need to get my data 

In [398]:
import os
import pandas as pd
import numpy as np
import mypytable as mypy
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [399]:
stockfiles= os.listdir("/home/cbarker4/Documents/DataScience/StockTrader/Data")
print(stockfiles)

['AADI.csv', 'AAPL.csv', 'ABCL.csv', 'ABIO.csv', 'AACG.csv', 'AAL.csv', 'ABEO.csv', 'AAON.csv', 'AACI.csv', 'AAME.csv', 'ABCM.csv', 'ABCB.csv', 'AACIW.csv', 'ABNB.csv', 'AAOI.csv']


In [400]:
X = []
Y=[]
Xlast=[]
for val in stockfiles:
    print(val)

    mt = mypy.MyPyTable()
    mt.load_from_file("/home/cbarker4/Documents/DataScience/StockTrader/Data/"+stockfiles[0])
    mt.drop_column('v')
    mt.drop_column('t')
    mt.drop_column('s')
    i = 100
    while i <  + len(mt.data)-1:
        table = mt.create_sub_table(i-100,i)

        table.drop_column('v')
        table.drop_column('t')
        X.append(table.data)
        Xlast.append(table.data[-1])
        Y.append(mt.get_row(i+1)[0])
        i+=1


AADI.csv
AAPL.csv
ABCL.csv
ABIO.csv
AACG.csv
AAL.csv
ABEO.csv
AAON.csv
AACI.csv
AAME.csv
ABCM.csv
ABCB.csv
AACIW.csv
ABNB.csv
AAOI.csv


In [401]:
X = np.array(X)
print(Y)
Y = np.array(Y)
# print(Y.shape)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.25 ,random_state=0)


[101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 151.0, 152.0, 153.0, 154.0, 155.0, 156.0, 157.0, 158.0, 159.0, 160.0, 161.0, 162.0, 163.0, 164.0, 165.0, 166.0, 167.0, 168.0, 169.0, 170.0, 171.0, 172.0, 173.0, 174.0, 175.0, 176.0, 177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 183.0, 184.0, 185.0, 186.0, 187.0, 188.0, 189.0, 190.0, 191.0, 192.0, 193.0, 194.0, 195.0, 196.0, 197.0, 198.0, 199.0, 200.0, 201.0, 202.0, 203.0, 204.0, 205.0, 206.0, 207.0, 208.0, 209.0, 210.0, 211.0, 212.0, 213.0, 214.0, 215.0, 216.0, 217.0, 218.0, 219.0, 220.0, 221.0, 222.0, 223.0, 224.0, 225.0, 226.0, 227.0, 228.0, 229.0, 230.0, 231.0, 232.0, 233.0, 234.0, 235.0, 236.0, 237.0, 238.0, 239.0, 240.0, 241.0, 242.0, 243.0

In [402]:
# Check shapes
print('X_train shape: ', X_train.shape)
print('y_train shape: ', Y_train.shape)
print('X_test shape: ', X_test.shape)
print('y_test shape: ', Y_test.shape)

X_train shape:  (1687, 100, 5)
y_train shape:  (1687,)
X_test shape:  (563, 100, 5)
y_test shape:  (563,)


In [403]:
def accuracy(y_true,y_pred):

    good = abs(y_true-y_pred)
    if good>.25:
        False
    else:
        True
    


# Build epoch checkpoint callback
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor = 'loss',
        patience = 7,
        verbose = 1,
        min_delta = 0,
        mode = 'min',
        baseline = None,
        restore_best_weights = True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join("/home/cbarker4/Documents/DataScience/StockTrader/Model", 'ckpt', "{epoch:02d}-{val_loss:.2f}.hdf5"),
        monitor = 'loss',
        verbose = 1,
        save_best_only = True,
        save_weights_only = False,
        mode = 'min',
        save_freq = 'epoch',
        options = None,
        initial_value_threshold = None
    )   
]

In [404]:


model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(units=64,
                            return_sequences=True,
                            input_shape=(X_train.shape[1], 5)))
model.add(tf.keras.layers.LSTM(units=64))
model.add(tf.keras.layers.Dense(32))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1))
model.summary


model.compile(optimizer='adam',loss = 'mean_squared_error')#,metrics=accuracy)
history = model.fit(X_train,Y_train,epochs = 500, 
                                      validation_split = 0.2, 
                                      callbacks = callbacks)
  

Epoch 1/500
43/43 [==============================] - ETA: 0s - loss: 28098.7871
Epoch 1: loss improved from inf to 28098.78711, saving model to /home/cbarker4/Documents/DataScience/StockTrader/Model/ckpt/01-22581.60.hdf5
43/43 [==============================] - 2s 23ms/step - loss: 28098.7871 - val_loss: 22581.6016
Epoch 2/500
41/43 [===========================>..] - ETA: 0s - loss: 16617.2656
Epoch 2: loss improved from 28098.78711 to 16458.31445, saving model to /home/cbarker4/Documents/DataScience/StockTrader/Model/ckpt/02-11597.86.hdf5
43/43 [==============================] - 1s 22ms/step - loss: 16458.3145 - val_loss: 11597.8564
Epoch 3/500
43/43 [==============================] - ETA: 0s - loss: 7893.1821
Epoch 3: loss improved from 16458.31445 to 7893.18213, saving model to /home/cbarker4/Documents/DataScience/StockTrader/Model/ckpt/03-4882.08.hdf5
43/43 [==============================] - 1s 23ms/step - loss: 7893.1821 - val_loss: 4882.0825
Epoch 4/500
41/43 [===================

In [405]:

y_pred = model.predict(X_test)
print(y_pred)

18/18 [==============================] - 0s 4ms/step
[[105.02263 ]
 [228.83878 ]
 [146.18758 ]
 [182.07492 ]
 [182.07492 ]
 [228.78853 ]
 [218.7665  ]
 [185.59251 ]
 [206.54027 ]
 [212.7458  ]
 [225.15077 ]
 [147.53578 ]
 [146.18758 ]
 [166.87785 ]
 [180.04247 ]
 [198.66081 ]
 [151.2342  ]
 [105.02263 ]
 [220.0023  ]
 [145.10628 ]
 [108.05307 ]
 [106.12062 ]
 [147.58861 ]
 [ 93.25402 ]
 [146.65904 ]
 [229.11568 ]
 [126.39496 ]
 [219.33362 ]
 [222.02946 ]
 [126.79741 ]
 [183.18816 ]
 [142.81654 ]
 [108.05307 ]
 [229.11568 ]
 [105.7697  ]
 [183.18816 ]
 [188.51015 ]
 [173.71535 ]
 [141.64577 ]
 [168.49837 ]
 [220.0023  ]
 [106.14955 ]
 [141.64577 ]
 [111.04318 ]
 [167.3799  ]
 [167.76991 ]
 [221.33087 ]
 [203.6577  ]
 [173.22316 ]
 [166.87785 ]
 [140.77705 ]
 [169.40147 ]
 [209.6529  ]
 [181.66795 ]
 [147.49214 ]
 [145.10628 ]
 [226.71733 ]
 [227.57524 ]
 [227.57524 ]
 [223.12602 ]
 [219.33362 ]
 [147.51321 ]
 [151.2342  ]
 [229.05208 ]
 [146.65904 ]
 [146.18758 ]
 [106.11347 ]
 [106.149

In [406]:
Xlast=[]
correct = 0
for val in X_test:
    Xlast.append(val[-1][0])


for i,val in enumerate(Y_test):
    
    if Xlast[i] > val:
        if Xlast[i]>y_pred[i]:
            correct+=1
    else:
        if Xlast[i]<y_pred[i]:
            correct+=1 

print('Correct predictions: ', correct)
print('Incorrect predictions: ', len(y_pred) - correct)
print('Accuracy: ', correct / len(y_pred))

Correct predictions:  109
Incorrect predictions:  454
Accuracy:  0.1936056838365897
